challenges

- how to chunk the pdf document 
    - ideally would be by recipe - but how 

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
# pip install langchain openai faiss-cpu

from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
import os

model_name = "gpt-4o-mini"

# Set your OpenAI API key
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"

# Step 1: Load and preprocess data from PDF
def load_data_from_pdf(file_path):
    loader = PyPDFLoader(file_path)
    documents = loader.load()
    return documents

# Step 2: Create vector store for retrieval
def create_vector_store(documents):
    text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    processed_texts = []

    for doc in documents:
        processed_texts.extend(text_splitter.split_text(doc.page_content))

    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(processed_texts, embeddings)
    
    return vector_store

# Step 3: Build the RAG Chain
def create_rag_chain(vector_store):
    retriever = vector_store.as_retriever()
    
    prompt_template = PromptTemplate(
        input_variables=["context", "question"],
        template=(
            "You are a helpful recipe assistant. Based on the following recipe context: \n{context}\n\n"
            "Answer the user's question: \n{question}"
        ),
    )

    llm = ChatOpenAI(temperature=0, model=model_name)

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        chain_type="stuff",
        retriever=retriever,
        return_source_documents=True,
        chain_type_kwargs={"prompt": prompt_template},
    )
    
    return qa_chain

# Step 4: Main Functionality
def main():
    pdf_path = "USU-Student-Cookbook.pdf"

    # Load data from PDF and create vector store
    documents = load_data_from_pdf(pdf_path)
    vector_store = create_vector_store(documents)

    # Create RAG chatbot
    rag_chain = create_rag_chain(vector_store)

    print("Recipe Chatbot ready! Type 'exit' to stop.")

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Goodbye!")
            break

        response = rag_chain.run(user_input)
        print(f"Bot: {response['result']}")

In [21]:
# !pip install --upgrade pymupdf

In [48]:
import pymupdf
# fine but then you lose the formatting from this # 
def extract_text_from_pdf(pdf_path):
    doc = pymupdf.open(pdf_path)
    text = ""
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        text += page.get_text("blocks")
    return text


In [49]:
# from langchain_text_splitters import RecursiveCharacterTextSplitter
# MarkdownHeaderTextSplitter()

# could exploit page change to indicate where to chunk : \n3, \n4 ... 

pdf_text = extract_text_from_pdf('USU-Student-Cookbook.pdf')

# splitter = RecursiveCharacterTextSplitter(
#     separators=["\n \n \n \n", ],   # i.e. split document into chunks 
#     chunk_size=1000,
#     chunk_overlap=200,
#     length_function=len
# )

TypeError: can only concatenate str (not "list") to str

In [51]:
doc = pymupdf.open('USU-Student-Cookbook.pdf')
for page_num in range(len(doc)):
    page = doc.load_page(page_num)
    print(page.get_text("blocks"))
# doc.page_count

[(191.89451599121094, 135.14553833007812, 419.7187194824219, 154.11752319335938, 'UTAH STATE UNIVERSTIY\n', 0, 0), (86.71873474121094, 268.4113464355469, 526.5850830078125, 285.8443603515625, 'COLLEGE FRIENDLY RECIPES WITH TIPS AND TRICKS \n', 1, 0), (64.75779724121094, 288.6613464355469, 543.9994506835938, 306.0943603515625, 'TO HELP YOU FEEL CONFIDENT COOKING AND SHOPPING\n', 2, 0), (99.77342224121094, 172.76678466796875, 530.6273803710938, 250.027587890625, 'Student Cook Book \n', 3, 0)]
[(72.00338745117188, 38.160003662109375, 542.495849609375, 62.63566970825195, '2 \n \n', 0, 0), (72.00338745117188, 731.2732543945312, 74.4992446899414, 755.7489624023438, ' \n \n', 1, 0), (71.99999237060547, 74.4000244140625, 510.07293701171875, 115.6800537109375, 'Introduction \n \n \nFood. It’s something we all think about every day, even multiple times a day. It’s a \n', 2, 0), (71.99996948242188, 125.64007568359375, 516.5494384765625, 137.64007568359375, 'necessity. It’s a delicacy. It’s energy

In [ ]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import PromptTemplate
# from langchain.text_splitter import ParagraphTextSplitter
from langchain.document_loaders import PyPDFLoader
import os

def load_data_from_pdf(file_path):
    loader = PyPDFLoader("USU-Student-Cookbook.pdf")
    documents = loader.load()
    return documents

# Step 2: Split document by bold headings and create vector store
def create_vector_store(documents):
    # Regex to match bold headings (assuming headings are marked in a recognizable pattern)
    text_splitter = RegexTextSplitter(
        pattern=r"\n\s*\b[A-Z][A-Z ]+\b\s*\n",  # Adjust based on your document's heading format
        chunk_size=1000,
        chunk_overlap=100
    )
    processed_texts = []

    for doc in documents:
        processed_texts.extend(text_splitter.split_text(doc.page_content))

    embeddings = OpenAIEmbeddings()
    vector_store = FAISS.from_texts(processed_texts, embeddings)
    
    return vector_store

In [12]:
# !pip install pypdf

In [13]:
dat = load_data_from_pdf(file_path='')
vec = create_vector_store(dat)

NameError: name 'RegexTextSplitter' is not defined